## Barrel shifter (cyhdl library)

This variant uses mixed myIRL and cythonized myhdl code segments. Note that the procedural generation is quite a bit faster.

We import the barrelshifter library:

In [1]:
from library.barrelshifter import *

Compiling /tmp/_cython_inline_08cfa26591b7492f5757aa87dc70c928c0d51984.pyx because it changed.
[1/1] Cythonizing /tmp/_cython_inline_08cfa26591b7492f5757aa87dc70c928c0d51984.pyx


[Edit barrelshifter.py](library/barrelshifter.py)

Then we create a barrel shifter generator with data width 32 (power 5):

In [2]:
W_POWER = 5

TEST_VALUES = [
    (0xdead, 8, 0xdead << 8),
    (0x8f01, 15, 0x8f01 << 15),
]

b = BarrelShifterGenerator(W_POWER)

DEBUG CALLING CYTHONIZED CODE


Then we instance it first time in the test bench below. Note the instancing time on this first run.

In [3]:
from myirl.test.common_test import gen_osc
from myirl.emulation.myhdl import *

@block
def top_bs(b):
    clk = myirl.ClkSignal()
    ce = Signal(bool())
    val, result = [ Signal(intbv(0xaa00)[2 ** W_POWER:]) for i in range(2) ]
    s = Signal(intbv()[W_POWER:])
    
    inst = [
        b.barrel_shifter(clk, ce, val, s, result, False),
        gen_osc(clk, 2)
    ]


    @instance
    def stim():
        for item in TEST_VALUES:
            ce.next = False
            s.next = item[1]
            val.next = item[0]
            yield(clk.posedge)
            ce.next = True
            yield(clk.posedge)
            yield(clk.posedge)

            print(result)
            assert result == item[2]
            
        raise StopSimulation

    
    inst += [ stim ]
    return inst
 
@utils.timer
def test(b):    
    return top_bs(b)

design = test(b)

 DEBUG CREATE wrapper module for top_bs (EmulationModule 'top_top_bs') 
 Declare obj 'barrel_shifter' in context '(LIB: BarrelShifterGenerator 'bs')' 
 Declare obj 'shifter_stage' in context '(LIB: BarrelShifterGenerator 'bs')' 
 Declare obj 'cshift' in context '(LIB: BarrelShifterGenerator 'bs')' 
 Module bs: Existing implementation cshift, rename to cshift_1 
 Module bs: Existing implementation shifter_stage, rename to shifter_stage_1 
 Module bs: Existing implementation shifter_stage, rename to shifter_stage_2 
 Module bs: Existing implementation shifter_stage, rename to shifter_stage_3 
 Module bs: Existing implementation shifter_stage, rename to shifter_stage_4 
Creating process '_run/assign' with sensitivity (clk'rising,)
Creating sequential 'top_bs/stim' 
Finished test in 0.2729 secs


Running the test bench again, the duration is a little less than on the first run, because this module was already compiled.

In [4]:
design = test(b)

Finished test in 0.0129 secs


Finally, the quick & dirty verification:

In [5]:
from myirl import targets
from myirl.test.common_test import run_ghdl

f = design.elab(targets.VHDL, elab_all = True)
f += b.elab(targets.VHDL)
# print(f)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = True)

 Elaborating component top_bs__BarrelShifterGenerator 
 Writing 'top_bs' to file /tmp/myirl_top_top_bs_35onkwox/top_bs.vhdl 
 Creating library file /tmp/myirl_module_defs_7af5m0ry/module_defs.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_16_s1_0_0 
 Writing 'shifter_stage_4' to file /tmp/myirl_bs_5ry08z3w/shifter_stage_4.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_8_s1_0_0 
 Writing 'shifter_stage_3' to file /tmp/myirl_bs_5ry08z3w/shifter_stage_3.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_4_s1_0_0 
 Writing 'shifter_stage_2' to file /tmp/myirl_bs_5ry08z3w/shifter_stage_2.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_2_s1_0_0 
 Writing 'shifter_stage_1' to file /tmp/myirl_bs_5ry08z3w/shifter_stage_1.vhdl 
 Elaborating component cshift__BarrelShifterGenerator_s1_s1_s1_s1_s1_0_0_1 
 Writing 'cshift_1' to file /tmp/myirl_bs_5ry08z3w/cshift_1.vhdl 
 Elaboratin

0

In [6]:
# !cat -n /tmp/barrel_shifter.vhdl